In [1]:
import sys

sys.path.append('/home/shuo/Dropbox/projects/spiders/oa_spider')

from utils import recent_years, sort_by_value
from db import MongoManager

mongo_manager = MongoManager()

account_1 = {
    "id": 'huangshuo',
    "password": 'tjmc12345',
}
account_2 = {
    "id": 'lihongwei',
    "password": 'cmcc1234.',
}

collection_toread = account_2['id'] + '_toread'
collection_labels = account_2['id'] + '_labels'
collection_raw_train_data = account_2['id'] + '_raw_train_data'


def get_labels_dict():
    """
    Get group-label mapping
    :return: Such as {group:label}
    """
    labels_dict = {}
    for label in mongo_manager.find(collection_labels):
        for group in label['groups']:
            labels_dict[group] = label['name']
    return labels_dict


In [2]:
# In this task,we interested in jobs which waiting_for_read_tag is True
# But for train data integrity,we labeled all jobs ont only waiting-for-read
# If the job is not waiting-for-read,we labeled its readers with 'NOBODY'

# query = {
#     'job_type': '部门收文',
#     'waiting_for_read_tag': True
# }

persist_to = collection_toread
account = account_2
for year in recent_years():
    # jobs = process_raw_data(
    #     account=account_2['id'],
    #     year=year,
    #     query=query,
    # )
    jobs = mongo_manager.find(account['id'] + '_' + str(year))
    for job in jobs:
        refined = dict()
        refined['title'] = job['title']
        refined['job_id'] = job['job_id']
        # Some jobs' readers is [],so must check if job['readers'] is null or not
        if 'waiting_for_read_tag' in job.keys() \
                and job['readers']:
            # De-duplication,sort and serialization
            refined['readers'] = ['|'.join(sorted(set(job['readers'])))
                                  if len(job['readers']) > 1
                                  else job['readers'][0]]
        else:
            refined['readers'] = 'NOBODY'
        mongo_manager.insert(
            collection=persist_to,
            value=refined
        )

mongo_manager.count(persist_to)


31241

In [3]:
grouped_jobs = {}

groups, count = \
    mongo_manager.distinct(collection_toread, 'readers')
for group in groups:
    query = {'readers': group}
    grouped_jobs[group] = mongo_manager.count(
        collection_toread, query)
    # print("{group}: {count}".format(
    #     group=group,
    #     count=grouped_jobs[group]
    # ))
top_readers = sort_by_value(d=grouped_jobs)


In [5]:
# To check given readers read what
query = {'readers': '阚建杰'}
jobs = mongo_manager.find(collection_toread, query)
count = 0
for job in jobs:
    count += 1
    print(job)
count


In [3]:
# To organize interested labels
label_trans = {
    'name': 'UNK',
    'groups': [],
    'description': '未知分组',
}
mongo_manager.upsert_one(
    collection=collection_labels,
    conditions={'name': label_trans['name']},
    value=label_trans
)
###################################
label_trans = {
    'name': 'transmission',
    'groups': ['吴铭峰', '李林宪|王云山|王志宽|蔡向阳'],
    'description': '传输组',
}
mongo_manager.upsert_one(
    collection=collection_labels,
    conditions={'name': label_trans['name']},
    value=label_trans
)
###################################
# label_general = {
#     'name': 'general_manager',
#     'groups': ['王云山', '阚建杰'],
#     'description': '总经理',
# }
# mongo_manager.upsert_one(
#     collection=collection_labels,
#     conditions={'name': label_general['name']},
#     value=label_general
# )
###################################
label_wireless = {
    'name': 'wireless',
    'groups': ['景昊|王乐3|王争|陈奇志', '景昊|王乐3|王争|王云山|陈奇志'],
    'description': '无线组（不需要过总经理的文）',
}
mongo_manager.upsert_one(
    collection=collection_labels,
    conditions={'name': label_wireless['name']},
    value=label_wireless
)
###################################
# label_wireless_with_mgr = {
#     'name': 'wireless_with_manager',
#     'groups': ['景昊|王乐3|王争|王云山|陈奇志'],
#     'description': '无线组（需要过总经理的文）',
# }
# mongo_manager.upsert_one(
#     collection=collection_labels,
#     conditions={'name': label_wireless_with_mgr['name']},
#     value=label_wireless_with_mgr
# )
###################################
label_infrastructure = {
    'name': 'infrastructure',
    'groups': ['杨增宝'],
    'description': '基础配套组',
}
mongo_manager.upsert_one(
    collection=collection_labels,
    conditions={'name': label_infrastructure['name']},
    value=label_infrastructure
)


In [ ]:
labels_dict = get_labels_dict()
persist_to = collection_raw_train_data
for job in mongo_manager.find(collection_toread):
    value = dict()
    value['title'] = job['title']
    value['job_id'] = job['job_id']
    if job['readers'][0] in labels_dict.keys():
        value['label'] = labels_dict[job['readers'][0]]
    else:
        value['label'] = 'UNK'  # labels_dict[0]
    mongo_manager.insert(
        collection=persist_to,
        value=value
    )


In [8]:
# Count labeled data
mongo_manager.count(collection_raw_train_data,
                    {'label': {'$ne': 'UNK'}})


8970